In [10]:
import json
import pandas as pd
from pprint import pprint

In [11]:
data = json.load(open('data/dogs1.json'))
pprint(data)

{'animals': [{'_links': {'organization': {'href': '/v2/organizations/ma224'},
                         'self': {'href': '/v2/animals/48752074'},
                         'type': {'href': '/v2/types/dog'}},
              'age': 'Baby',
              'attributes': {'declawed': None,
                             'house_trained': False,
                             'shots_current': True,
                             'spayed_neutered': True,
                             'special_needs': False},
              'breeds': {'mixed': True,
                         'primary': 'Redbone Coonhound',
                         'secondary': 'American Bulldog',
                         'unknown': False},
              'coat': 'Short',
              'colors': {'primary': 'Red / Chestnut / Orange',
                         'secondary': None,
                         'tertiary': None},
              'contact': {'address': {'address1': None,
                                      'address2': None,
            

                         'self': {'href': '/v2/animals/48752050'},
                         'type': {'href': '/v2/types/dog'}},
              'age': 'Baby',
              'attributes': {'declawed': None,
                             'house_trained': False,
                             'shots_current': True,
                             'spayed_neutered': True,
                             'special_needs': False},
              'breeds': {'mixed': True,
                         'primary': 'Redbone Coonhound',
                         'secondary': 'American Bulldog',
                         'unknown': False},
              'coat': 'Short',
              'colors': {'primary': 'Red / Chestnut / Orange',
                         'secondary': None,
                         'tertiary': None},
              'contact': {'address': {'address1': None,
                                      'address2': None,
                                      'city': 'Andover',
                                 

                          'medium': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/48752053/2/?bust=1597187539&width=300',
                          'small': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/48752053/2/?bust=1597187539&width=100'},
                         {'full': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/48752053/4/?bust=1597187551',
                          'large': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/48752053/4/?bust=1597187551&width=600',
                          'medium': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/48752053/4/?bust=1597187551&width=300',
                          'small': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/48752053/4/?bust=1597187551&width=100'},
                         {'full': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/48752053/5/?bust=1597187574',
                          'large': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/48752053/5/?bust=1597187574&width=600',
                  

              'photos': [{'full': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/48752020/1/?bust=1597187791',
                          'large': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/48752020/1/?bust=1597187791&width=600',
                          'medium': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/48752020/1/?bust=1597187791&width=300',
                          'small': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/48752020/1/?bust=1597187791&width=100'},
                         {'full': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/48752020/3/?bust=1597187792',
                          'large': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/48752020/3/?bust=1597187792&width=600',
                          'medium': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/48752020/3/?bust=1597187792&width=300',
                          'small': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/48752020/3/?bust=1597187792&width=100'},
                  

              'gender': 'Male',
              'id': 48751976,
              'name': 'Humphrey',
              'organization_animal_id': '45192618',
              'organization_id': 'FL110',
              'photos': [{'full': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/48751976/1/?bust=1597187773',
                          'large': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/48751976/1/?bust=1597187773&width=600',
                          'medium': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/48751976/1/?bust=1597187773&width=300',
                          'small': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/48751976/1/?bust=1597187773&width=100'},
                         {'full': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/48751976/2/?bust=1597187772',
                          'large': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/48751976/2/?bust=1597187772&width=600',
                          'medium': 'https://dl5zpyw5k3jeb.cloudfront.net/photos

...

              'coat': None,
              'colors': {'primary': 'Brown / Chocolate',
                         'secondary': None,
                         'tertiary': None},
              'contact': {'address': {'address1': '111 W. Hunting Park Ave.',
                                      'address2': None,
                                      'city': 'Philadelphia',
                                      'country': 'US',
                                      'postcode': '19140',
                                      'state': 'PA'},
                          'email': 'adopt@acctphilly.org',
                          'phone': '(267) 385-3800'},
              'description': 'Intake Type: (8/6) Stray\n'
                             '\n'
                             'Dog Meet:\n'
                             'Gouda Boy and Shy: https://youtu.be/VH5wcKOavUQ\n'
                             'Gouda Boy and Vulfpeck: '
                             'https://youtu.be/WrDwz6kHRz0',
              'di

                                        'large': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/48751821/1/?bust=1597186522&width=600',
                                        'medium': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/48751821/1/?bust=1597186522&width=450',
                                        'small': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/48751821/1/?bust=1597186522&width=300'},
              'published_at': '2020-08-11T22:45:11+0000',
              'size': 'Small',
              'species': 'Dog',
              'status': 'adoptable',
              'status_changed_at': '2020-08-11T22:45:11+0000',
              'tags': [],
              'type': 'Dog',
              'url': 'https://www.petfinder.com/dog/tusc-stray-tu1510-48751821/al/tuscaloosa/tuscaloosa-metro-animal-shelter-al52/?referrer_id=47af4814-b861-4759-a5e6-fa07193152f5',
              'videos': []},
             {'_links': {'organization': {'href': '/v2/organizations/in190'},
                

              'photos': [{'full': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/48749166/3/?bust=1597185858',
                          'large': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/48749166/3/?bust=1597185858&width=600',
                          'medium': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/48749166/3/?bust=1597185858&width=300',
                          'small': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/48749166/3/?bust=1597185858&width=100'},
                         {'full': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/48749166/1/?bust=1597176874',
                          'large': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/48749166/1/?bust=1597176874&width=600',
                          'medium': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/48749166/1/?bust=1597176874&width=300',
                          'small': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/48749166/1/?bust=1597176874&width=100'},
                  

              'description': 'How this pet arrived: This pet was brought in by '
                             'their previous owner. Why this pet is the one '
                             'for...',
              'distance': None,
              'environment': {'cats': None, 'children': None, 'dogs': None},
              'gender': 'Male',
              'id': 48751792,
              'name': 'Chewie',
              'organization_animal_id': '678795',
              'organization_id': 'CA448',
              'photos': [{'full': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/48751792/1/?bust=1597186262',
                          'large': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/48751792/1/?bust=1597186262&width=600',
                          'medium': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/48751792/1/?bust=1597186262&width=300',
                          'small': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/48751792/1/?bust=1597186262&width=100'}],
              '

                         'self': {'href': '/v2/animals/48751758'},
                         'type': {'href': '/v2/types/dog'}},
              'age': 'Young',
              'attributes': {'declawed': None,
                             'house_trained': False,
                             'shots_current': True,
                             'spayed_neutered': True,
                             'special_needs': False},
              'breeds': {'mixed': True,
                         'primary': 'Shepherd',
                         'secondary': 'Black Mouth Cur',
                         'unknown': False},
              'coat': None,
              'colors': {'primary': 'Yellow / Tan / Blond / Fawn',
                         'secondary': 'White / Cream',
                         'tertiary': None},
              'contact': {'address': {'address1': '4205 NW 6th Street',
                                      'address2': None,
                                      'city': 'Gainesville',
          

In [8]:
#list of dictionaries (dict starts at animal id)
#we want to extract id, name, breed (primary), contact(city and state)

In [19]:
# confirming that there are 100 adoptable dogs per page (json file)
len(data['animals'])

100

In [40]:
def petfinder(json_num):
    data = json.load(open(f'data/dogs{json_num}.json'))
    
    pet_list = []
    
    for i in range(len(data['animals'])): 
        breed = data['animals'][i]['breeds']['primary']
        city = data['animals'][i]['contact']['address']['city']
        state = data['animals'][i]['contact']['address']['state']
        pet_id = data['animals'][i]['id']
        name = data['animals'][i]['name']
    
        pet_dict={}

        pet_dict = {
            "breed": breed,
            "city": city,
            "state": state,
            "pet_id": pet_id,
            "name": name,
            }
    
        pet_list.append(pet_dict)
    
    return pet_list

In [44]:
list1 = list(petfinder(1))
list2 = list(petfinder(2))
list3 = list(petfinder(3))
list4 = list(petfinder(4))
list5 = list(petfinder(5))
all_dogs=list1+list2+list3+list4+list5

In [45]:
all_dogs

[{'breed': 'Redbone Coonhound',
  'city': 'Andover',
  'state': 'MA',
  'pet_id': 48752074,
  'name': 'Jade'},
 {'breed': 'Labrador Retriever',
  'city': 'New York',
  'state': 'NY',
  'pet_id': 48752069,
  'name': 'Wish'},
 {'breed': 'Shiba Inu',
  'city': 'TULSA',
  'state': 'OK',
  'pet_id': 48752062,
  'name': 'Molly'},
 {'breed': 'Retriever',
  'city': 'White Plains',
  'state': 'NY',
  'pet_id': 48751967,
  'name': 'Brindi'},
 {'breed': 'Greyhound',
  'city': 'Wasco',
  'state': 'CA',
  'pet_id': 48751842,
  'name': '(FRECKLES) AVAILABLE AUG 24 2020'},
 {'breed': 'Pit Bull Terrier',
  'city': 'Albuquerque',
  'state': 'NM',
  'pet_id': 48752073,
  'name': 'Posole'},
 {'breed': 'Catahoula Leopard Dog',
  'city': 'Corpus Christi',
  'state': 'TX',
  'pet_id': 48752072,
  'name': 'Alyx'},
 {'breed': 'Redbone Coonhound',
  'city': 'Andover',
  'state': 'MA',
  'pet_id': 48752050,
  'name': 'Jackson'},
 {'breed': 'Chihuahua',
  'city': 'Columbus',
  'state': 'OH',
  'pet_id': 48752065

In [51]:
dogs_df = pd.DataFrame(all_dogs)
dogs_df

,breed,city,state,pet_id,name
0,Redbone Coonhound,Andover,MA,48752074,Jade
1,Labrador Retriever,New York,NY,48752069,Wish
2,Shiba Inu,TULSA,OK,48752062,Molly
3,Retriever,White Plains,NY,48751967,Brindi
4,Greyhound,Wasco,CA,48751842,(FRECKLES) AVAILABLE AUG 24 2020
...,...,...,...,...,...
495,Pit Bull Terrier,Dallas,TX,48750352,1105763
496,Mixed Breed,Santa Fe,NM,48750344,ZUMA
497,Maltese,Santa Cruz,CA,48750345,MUPPET*
498,Mixed Breed,Santa Fe,NM,48750342,JEREMIAH


In [50]:
breed_count = pd.DataFrame(dogs_df['breed'].value_counts())
breed_count

,breed
Labrador Retriever,66
Pit Bull Terrier,56
Mixed Breed,41
Chihuahua,41
German Shepherd Dog,26
...,...
Norfolk Terrier,1
Pekingese,1
Papillon,1
English Pointer,1


In [52]:
dogs_df.to_csv("data/petfinder_dogs.csv", index=False)